In [ ]:
# !pip install llm-comparator

In [ ]:
from llm_comparator import model_helper
from llm_comparator import llm_judge_runner

In [ ]:
#@title Setup for using Vertex AI.
from google.colab import auth
auth.authenticate_user()

PROJECT_ID = 'pair-experimental'  #@param {type: "string"}
REGION = 'us-central1'  #@param {type: "string"}

! gcloud config set project {PROJECT_ID}

generator = model_helper.VertexModelHelper(project_id=PROJECT_ID, region=REGION)

In [ ]:
llm_judge_inputs = [
    {'prompt': 'how are you?', 'response_a': 'good', 'response_b': 'bad'},
    {'prompt': 'hello?', 'response_a': 'hello', 'response_b': 'hi'},
    {'prompt': 'what is the capital of korea?', 'response_a': 'Seoul', 'response_b': 'Vancouver'}
]

In [ ]:
# Run LLM judge.
judge = llm_judge_runner.LLMJudgeRunner(generator)
outputs = judge.run(llm_judge_inputs, 4)

# TODO: Create a wrapper class that includes both LLM judge and rationale summary (not implemented yet).

Created 12 inputs for LLM judge.


  0%|          | 0/12 [00:00<?, ?it/s]

Generated 12 outputs from LLM judge.
Parsed 3 example ratings.
Generated ratings for 3 examples.


In [ ]:
#@title Prepare JSON for LLM Comparator
# TODO: Move to the pip package.
import json

llm_comparator_data = {
    'metadata': {'custom_fields_schema': []},
    'models': [{'name': 'A'}, { 'name': 'B'}],
    'examples': [{
        'input_text': input['prompt'],
        'tags': [],
        'output_text_a': input['response_a'],
        'output_text_b': input['response_b'],
        'score': output['score'],
        'individual_rater_scores': output['individual_ratings']
    } for input, output in zip(llm_judge_inputs, outputs)]
}

with open('json_for_llm_comparator.json', 'w') as f:
  json.dump(llm_comparator_data, f)

In [ ]:
!git clone https://github.com/PAIR-code/llm-comparator

In [ ]:
#@title For displaying LLM Comparator.
from IPython.display import Javascript

# TODO: Move to the pip package.
def show_llm_comparator(json_path, height=800, port=8888):
  get_ipython().system_raw(f'python3 -m http.server {port} &')
  display(Javascript("""
  (async ()=>{
    fm = document.createElement('iframe')
    fm.src = await google.colab.kernel.proxyPort(%s)
    results_path = fm.src + '%s'
    fm.src += 'llm-comparator/docs/'
    fm.src += '?results_path=' + results_path
    fm.width = '100%%'
    fm.height = '%d'
    fm.frameBorder = 0
    document.body.append(fm)
  })();
  """ % (port, json_path, height) ))

In [ ]:
show_llm_comparator('json_for_llm_comparator.json')

<IPython.core.display.Javascript object>